In [1]:
import pandas as pd


In [2]:
CSV_FILE_PATH = './sample_taxi/sample_taxi.csv'
df = pd.read_csv(CSV_FILE_PATH)
df.head()

,taxi_id,time,lon,lat,is_passenger,speed
0,22224,10:13:42,113.887032,22.547518,0,0
1,22224,19:31:36,113.909554,22.582199,1,71
2,22224,17:06:32,113.916748,22.623550,1,100
3,22224,15:07:19,113.922432,22.535383,1,21
4,22224,14:55:18,113.922668,22.520033,1,10


In [ ]:
taxi_counts = dict(df.taxi_id.value_counts())

In [ ]:
id_list = list(taxi_counts.keys())

In [8]:
OD_list = []
OD_path = []
for id in id_list:
    id_df = df[df.taxi_id == id]
    id_df = id_df.sort_values(by='time')
    passenger_check = False

    if (id_df.iloc[0]['is_passenger'] == 0):
        passenger_check = False
    else:
        passenger_check = True
    for index, row in id_df.iterrows():
        if (passenger_check and row['is_passenger'] == 0):
            passenger_check = False
            if (len(OD_path) == 1):
                OD_path.append([row['time'], row['lon'], row['lat']])
                OD_list.append(OD_path.copy())
                OD_path = []
        elif ((not passenger_check) and row['is_passenger'] == 1):
            passenger_check = True
            OD_path = []
            OD_path.append([row['time'], row['lon'], row['lat']])

In [9]:
r_df1 = pd.DataFrame(
    columns=['from_time','lng1','lat1','to_time','lng2','lat2'])
for i in OD_list:
    slide_series = pd.Series(
        {'from_time': i[0][0],
         'lng1': i[0][1],
         'lat1': i[0][2],
         'to_time': i[1][0],
         'lng2': i[1][1],
         'lat2': i[1][2]})
    r_df1 = r_df1.append(slide_series, ignore_index=True)
r_df1.to_csv("./data/all_data.csv", index=False, sep=',')